Before you turn this problem in, make sure everything runs as expected. First, **restart the kernel** (in the menubar, select Kernel$\rightarrow$Restart) and then **run all cells** (in the menubar, select Cell$\rightarrow$Run All).

Make sure you fill in any place that says `YOUR CODE HERE` or "YOUR ANSWER HERE", as well as your name and collaborators below:

In [4]:
NAME = "Martijn van de Wouw"
STUDENT_NUMBER = "s1052099"
COLLABORATOR_NAME = "Joost Grunwald"
COLLABORATOR_STUDENT_NUMBER = "s1057493"

---

## **Important:** When handing in your homework:
+ Hand in the notebook (and nothing else) **named as follows**: StudentName1_snumber_StudentName2_snumber.ipynb
+ Provide clear and complete answers to the questions below under a separate header (not hidden somewhere in your source code), and make sure to explain your answers / motivate your choices. Add Markdown cells where necessary.
+ Source code, output graphs, derivations, etc., should be included in the notebook.
+ Hand-in: upload to Brightspace.
+ Include name, student number, assignment (especially in filenames)!
+ When working in pairs only one of you should upload the assignment, and report the name of your partner in your filename.
+ Use the Brightspace discussion board or email the student assistants for questions on how to complete the exercises.
+ If you find mistakes/have suggestions/would like to complain about the assigment material itself, please email me [Roel] at `Roel.Bouman@ru.nl`
+ Do not add/remove any cells in the notebook, else this might break the auto-grading system. **An invalid notebook will mean a severe reduction in your grade!**
+ Only type your answers in those places where they are asked.
+ Remove any "raise NotImplementedError()" statements in the cells you answered.

# Assignment 6: Nearest Neighbour and Artificial Neural Networks

## Objective of this assignment
The objective of this exercise is to understand how k-nearest neighbor and neural networks can be used to solve classification problems.


## Advised Reading and Exercise Material
**The following reading material is recommended:**

- Pang-Ning Tan, Michael Steinbach, and Vipin Kumar, *Introduction to Data Mining*, section 5.2-5.4.


## Additional Tools
This exercise is based upon material kindly provided by the Cognitive System Section, DTU Compute, http://cogsys.compute.dtu.dk. Any sale or commercial distribution is strictly forbidden.


##  6.1 K-nearest neighbor classification
In this exercise we will use the k-nearest neighbors (KNN) method for classification.
First, we will consider the four synthetic data sets synth1, synth2, synth3 and
synth4 we used in earlier assignments.

#### 6.1.1 (2 points)
For each of the four synthetic data sets, do the following.:

* Load the training part of the dataset `X_train` and `y_train` as well as `X_test` and `y_test`.

* Fit a  k-nearest neighbor classifier model (`KNeighborsClassifier` from `sklearn.neighbors` (http://scikit-learn.org/stable/modules/generated/sklearn.neighbors.KNeighborsClassifier.html)) on the train data. 
  * Choose a suitable distance measure (you should consider the distance measures `euclidean`,`cityblock`, `cosine`, and `seuclidean`).
    * For the `seuclidean` distance you need to supply an additional parameter `V`. `V` should be a **vector** containing the variance of each variable in `X`.

  * Choose a suitable number of neighbors. 
  
* Predict the class of the test data using the trained model.

* Make a scatterplot of the train and test data with the classification of the test data obtained with the best k-value and distance measures you found -- just one plot per data set is fine. You can use the `classification_plot` function from the toolbox. 
  * Use the obtained prediction of the test data in your plot.
  * Try to study the plot (use all 5 arguments) to see how test data is classified using the train data.

* Create the confusion matrix, plot it using the `plot_confusion_matrix` function given below, and calculate the accuracy and error rate. Print or show the accuracy and error rate for each dataset.

Answers the following questions for each dataset:

* Which distance measures worked best for each of the four problems? Can you explain why?

* How many neighbors were needed for the four problems? 

* Can you give an example of when it would be good to use a large/small number of neighbors? Consider e.g. when clusters are  well separated versus when they are overlapping.

Hints:

* To generate a confusion matrix, you can use the function confusion_matrix() from the module sklearn.metrics. You can use the function below to plot the confusion matrix. Don't remember how to read a confusion matrix? Check the wiki page: https://en.wikipedia.org/wiki/Confusion_matrix



In [5]:
import seaborn as sn
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

def plot_confusion_matrix(cm,y):
    df_cm = pd.DataFrame(cm, index = [i+1 for i in np.unique(y)],
                  columns = [i+1 for i in np.unique(y)])
    plt.figure()
    sn.heatmap(df_cm, annot=True)
    plt.title('Confusion matrix')
    plt.xlabel('Predicted class')
    plt.ylabel('Actual class')
    plt.show()

In [7]:
import scipy.io as sc
from sklearn.neighbors import KNeighborsClassifier
from sklearn.neighbors import DistanceMetric
from sklearn.metrics import confusion_matrix
from toolbox.classification_plot import classification_plot as cp


def run(file, n, metric, show):
    #getting data
    data = sc.loadmat("./data/"+file)
    X_train = data["X_train"]
    X_test = data["X_test"]
    y_train = data["y_train"]
    y_test = data["y_test"]
    #needs extra paramater
    if (metric == 'seuclidean'):
        V = np.var(X_train, axis=0)#calculate extra parameter
        KNN = KNeighborsClassifier(n_neighbors=n, algorithm='auto', metric=metric, metric_params={'V':V})
    else:
        KNN = KNeighborsClassifier(n_neighbors=n, algorithm='auto', metric=metric)
    #train
    KNN.fit(X_train, y_train.ravel())

    #predict with test set
    predict = KNN.predict(X_test)
    
    #make confusion matrix
    cm = confusion_matrix(y_test, predict)
    t = 0
    #get amount of correct predictions
    for i in range(len(cm)):
        t += cm[i][i]
    if(show):
        plot = cp(X_test, y_test.ravel(), predict, X_train, y_train.ravel())
        plot_confusion_matrix(cm, y_test)
    #return values for other functions to use
    return t,len(y_test.ravel())
    
    
    
    
def getBestK(file):
    metrics = ['euclidean', 'cosine', 'manhattan', 'seuclidean']
    #best amount of correct predictions per metric
    best = 0
    #best amont of neighbors per metric
    ind = 0
    #size of test data
    size = 0
    #best of all metrics
    bestM = 0
    #the best metric
    M = ""
    #number of correct predictions of the best metric
    indM = 0
    
    
    for m in metrics:
        #loop over n neighbors to see amount which does best
        for i in range(1, 40):
            t,size = run(file, i, m, False)
            if (t>best):
                best = t
                ind = i
        if(best > bestM):
            bestM = best
            M = m
            indM = ind
        best = 0
        ind = 0
    #print the results of the best metric
    print("Distance metric: \t" +M)
    print("Correct predictions: \t" +str(bestM))
    print("\tAccuracy \t= " + str(bestM/size))
    print("\tError rate \t= " + str((size - bestM) / size))
    print("With k neighbors: \t" +str(indM))
    #show the plot and confusion matrix of best metric
    run(file, indM, M, True)
    
def runfiles():
    #go over all the files
    files = ["synth1.mat", "synth2.mat","synth3.mat","synth4.mat"]
    for i in files:
        print("CALCULATING FOR FILE \t" + i)
        print("=======================")
        getBestK(i)
        print("=======================")
        
runfiles()

ModuleNotFoundError: No module named 'toolbox'

# GIVE ANSWER

#### 6.1.2 (1.25 points)
In general we can use cross-validation to select the best parameters for our method.This can however be computationally expensive. 

We now return to the Iris data that we have considered in previous exercises, and will attempt to classify the Iris flowers using KNN. 

* Load the Iris data into Python with the `pandas` function `read_excel()` and save it to a variable called `iris_data`. Inspect the data by printing the `head()` of the dataframe. 

* Use the values of the 4 variables `Sepal Length,  Sepal Width,  Petal Length,  Petal Width` to create a data set `X`. Use the `Type` column to create the labels `labels`. Both `X` should be a numpy array! You can use the `to_numpy()` method for this. `labels` will be a Pandas series.

* Convert the Pandas series `labels` to an integer encoding by using the `LabelEncoder` in scikit-learn. Save the integer encoding, which should be a numpy array to a variable called `y`.

* Use leave-one-out cross-validation to estimate the optimal number of neighbors, k, for the k-nearest neighbor classifier. Save the vector of averaged errors to a variable called `mean_errors`. This should be a numpy array.

* Plot the cross-validation average classification error as a function of k for $k = 1,..,40$.

You can use the function `LeaveOneOut` from `sklearn.model_selection`: http://scikit-learn.org/stable/modules/generated/sklearn.model_selection.LeaveOneOut.html

What is the optimal number of neighbors to use for this data?

In [ ]:
from sklearn import preprocessing
from sklearn.model_selection import LeaveOneOut
import matplotlib.pyplot as plt
iris_data = pd.read_excel("data\iris.xls")
print(iris_data.head())
#prepare data
X = iris_data.iloc[:,0:3].to_numpy()
labels = iris_data.iloc[:,-1]
y = preprocessing.LabelEncoder().fit(labels).transform(labels)
#create splits
split = LeaveOneOut()
split.get_n_splits(X)
mean_errors = []
ns = list(range(1,150))
for i in range(40):
    err = []
    for train_index, test_index in split.split(X):
        #create data splits
        X_train = X[train_index]
        X_test = X[test_index]
        y_train = y[train_index]
        y_test = y[test_index]
        KNN = KNeighborsClassifier(n_neighbors = ns[i], algorithm='auto', metric = 'euclidean')
        #train
        KNN.fit(X_train, y_train)

        #predict with test set
        predict = KNN.predict(X_test)
        if(predict[0] == y_test[0]):
            err.append(0)
        else:
            err.append(1)
    errnp = np.asarray(err)

    mean_errors.append(errnp.mean())
#plot figure
plt.figure()
plt.plot(list(range(1,41)), np.asarray(mean_errors))
plt.xlabel('n neighbors')
plt.ylabel('averaged error')
plt.show()

In [ ]:
"""Checks whether 6.1.2 output is correct"""
"""DO NOT MODIFY THIS CELL"""

for this data it seems that around 11 neighbours gives the best result, it seems that when we use less we are more sensitive to outliers and when we use more we take too many data points into consideration.

## 6.2 Artificial Neural Networks

In this part of the exercise we will use neural networks to classify the xor data. We will consider a network with an input layer, one layer of hidden units and an output layer. The class `sklearn.neural_network.MLPClassifier` can be used to create a Multilayer Perceptron that can minimizes the Cross-Entropy loss function for any dataset X with corresponding labels y. See https://scikit-learn.org/stable/modules/neural_networks_supervised.html 

We use the data contained in `xor.mat` in the Data folder. 

#### 6.2.1 (0.5 points)
Check out the documentation for `MLPClassifier` and read the documentation well. Make sure you understand at least in general terms how the learning process works. Answer the following questions before you continue:

1. For a single perceptron, the activation function can be linear, e.g. $f(x) = x$. However, the activation function used in the MLP class is a non-linear function. Why does it not make sense for a MLP to use such a linear activation function? Hint: what does the following computation simplify to if $f(x)$ is such a simple linear function: $f(Wo*f(Wh*X_i))$?
2. The MLPClassifier has a few optional parameters. For each of the following parameters, explain how changing the parameter might affect the learning process or the resulting solution:

> `hidden_layer_sizes`:

> `max_iter`:

> `learning_rate`:

> `learning_rate_init`:

   3\. Use the following commands to create a small test set:

> `X = np.array([[0,0],[0,1],[1,0],[1,1]]).`

> `y = np.array([0,0,0,1])`
     

Then: 
- Create a MLPClassifier with 1 hidden layer containing 1 unit using the `lbfgs`solver and fit the data. You can leave the other parameters unchanged. Use the `score` method of the classifier object to compute the mean accuracy. How well does the MLP perform on this problem? Use the function `plot_boundaries` function from the toolbox to plot the data and the decision boundaries. Why does(n't) it work well with one hidden unit? Could you improve by using more?

#### NB: the weights are initialised randomly, so you should run the code a few times untill you get a decent result.

**Helpful hint:**
- You can plot decision boundaries as follows:

`plot_boundaries(X,y,clf)`


YOUR ANSWER HERE

In [ ]:
##Answer to question 6.2.1
# YOUR CODE HERE
raise NotImplementedError()


#### 6.2.2 loading and visualizing the Xor data (0.5)
In this exercise, we'll look at the Xor data in `xor.mat`.


- Load the data into Python using `scipio.io.loadmat(...)` and make `y` a 1d vector. Make a scatter plot of the two attributes in `X`, coloring the points according to the class labels `y` and add a legend. Use matplotlib functions, and no toolbox functions for this. 

In [ ]:
##Answer to question 6.2.2
# YOUR CODE HERE
raise NotImplementedError()

#### 6.2.3 Optimizing the number of hidden units (3 points) 
In this exercise, we'll start optimizing a MLPClassifier for use on the Xor data. We've toyed around a bit with finding the optimal number of hidden units, but now we'll optimize them in earnest.

- Create a CV loop for MLPClassifier with 1 to 20 hidden units. Fit X and y. Use 10-fold cross-validation `KFold` from `sklearn.model_selection` to create training and test sets and estimate the classification error for both. **In each fold, for each numberof hidden units, run the learning process 5 times and take the average classification error.**  
- Determine the optimal number of hidden units.
- Plot the decision boundaries of one network with the optimal number of hidden units trained on the entire data set, and explain why the network performs so well/poorly.

In [ ]:
##Answer to question 6.2.3
# YOUR CODE HERE
raise NotImplementedError()

YOUR ANSWER HERE

## 6.3 Support Vector Machines

In this subsection we're taking a look at Support Vector Machines.
We'll again analyze the Xor data. and look at what some of the hyperparameters do.

### 6.3.1 Linear kernels (1 point)
- First, use a linear kernel support vector machine from Scikit-learn, `SVC`, and train it on the Xor data. 
- Again, plot the boundaries of the classification using the `plot_boundaries` function from the toolbox
- What do you observe when you use the basic settings, but with a linear kernel? Why does or doesn't the model perform well? 
- Do you get different results when rerunning the algorithm?

In [ ]:
from sklearn.svm import SVC
from toolbox.plot_boundaries import plot_boundaries as pb
data = sc.loadmat("./data/xor.mat")
X = data['X']
y = data['y'].ravel()
svc = SVC(kernel = 'linear')
svc.fit(X, y)
pb(X, y, svc)

because of the linear model, svc is not able to make correct predictions for each cluster and thus has to 'choose' which cluster to predict wrongly. When rerunning the results stay the same

### 6.3.2 The kernel trick (1.25 points)
In order to solve the problem a linear kernel SVM has when facing such multimodal data, we can transform the data using a so called kernel. Several kernels exist, but in this case we'll look into a second degree polynomial kernel first.

There are several ways to construct a second degree polynomial kernel
- You can include or exclude the bias (the ^0 term)
- You can include or exclude non-interactions (X_1^d) terms

In this case, we can just add a third variable/feature to the data, namely the interaction term $x_1 * x_2$. So we don't include the $x_1^2$ and $x_2^2$ terms.

- Calculate the third interaction feature for the kernel. You can do this manually, or using the `PolynomialFeatures` function from Scikit-learn with the correct settings. Verify you end up with a $400*3$ matrix.
- Make a 3D scatterplot of the transformed data, color the points by the label and add a legend
- Use a linear kernel SVM with otherwise default settings to predict `y`. What is the accuracy of the classifier now? Does this match up with what you see in the plot? Can you find a separating hyperplane?

In [ ]:
from sklearn.preprocessing import PolynomialFeatures
poly = PolynomialFeatures(degree = 2, interaction_only=True)
data = poly.fit_transform(X)[:,1:4]
print(data.shape)

fig = plt.figure(figsize=(20,20))
ax = fig.add_subplot(projection='3d')
ax.scatter(data[(y==1),0],data[(y==1),1],data[(y==1),2],label='Label = 1', c="r", marker="o")
ax.scatter(data[(y==0),0],data[(y==0),1],data[(y==0),2],label='Label = 0', c="b", marker="o")
ax.legend()
svc.fit(data, y)
predict = svc.predict(data)
print(y[predict==y])
print(len(y))
print(len(y[predict==y]))

accuracy is 398/400 which is a bit less than 1000 %, I think it does match up as I do see a 2d plane (hyperplane) which can correctly separate the points. it would be a curved hyperplane which bends as a sort of dome over the points with label 1

### 6.3.3 Using the kernel trick directly (0.75 points)

Instead of manually applying the kernel trick, we can also use the polynomial kernel directly when initializing the SVC object. 

- Apply SVM using a second degree polynomial kernel and calculate and print the score.
- Use the `plot_boundaries` function to show the classification boundaries. 
- Is the result the same as when applying the trick manually? Why (not)?

In [ ]:
svc = SVC(degree = 2,kernel = 'poly')
svc.fit(data, y)
svc.predict(data)
predict = svc.predict(data)
score = svc.score(data, y)
#pb(data, y, svc)
#does not seem to work

it is, it probably has to do with the extra dimensenion the linear SVM gets which makes it have the same result as the polynomial one.